## Niger Little-Poole
## HW6

In [4]:
import math
import random


<B>1.</B> Here you will simulate the FIFO GI/GI/1 queue up to time T = 40 with the following
exponential rate λ = 1/5 distribution A(x) = P(X ≤ x) for interarrival times, and
uniform distribution over [2, 4], G(x) = P(S ≤ x), for service times:
$$ A(x) = 1 − e^{−\frac{1}{5}x}, x ≥ 0 $$
$$ G(x) = \frac {x − 2}{2}, x ∈ [2, 4] $$

You are to obtain a copy of both L(T) and $\frac{1}{T} \int^T_0 L(s) ds$ where ${L(t) : t ≥ 0}$ is thenumber of customers in the system process, and initially L(0) = 0.

In [6]:
def arrival(lamb):
    return (-1/lamb)*math.log(random.random())
def deperature(a):
    return a*random.random() + a

def fifo(T, lamb, a ):
    t =0
    t_a = arrival(lamb)
    t_d = math.pow(10,6)
    n = 0
    n_a = 0
    n_d = 0
    integral = 0
    while t_a < T and (t_d < T or t_d == math.pow(10,6)):
        if t_a < t_d:
            integral += n*(t_a - t)
            n_a+=1
            n+=1
            t = t_a
            t_a = arrival(lamb)+ t
            if n ==1:
                t_d = deperature(a) + t
        else:
            integral+= n*(t_d -t)
            n_d +=1
            n -= 1
            t = t_d
            if n ==0:
                t_d = math.pow(10,6)
            else:
                t_d = deperature(a) + t
    integral += n*(T-t)
    return n, integral/T
print fifo(40, .2,2)

(2, 0.6870118596838635)


<B>2.</B> Continuation: Use the above algorithm N = 10, 000 times so as to use Monte Carlo to estimate both $E(L(T))$, and $E[\frac{1}{T}\int^T_0L(S)ds]$



In [9]:
def monte_carlo_fifo(n, T, lamb, a):
    integrals =[]
    expected = []
    for i in range(n):
        ex, inte = fifo(T, lamb, a)
        integrals.append(inte)
        expected.append(ex)
    return 1.0*sum(expected)/n, sum(integrals)/n
print monte_carlo_fifo(10000,40, .2,2)

(1.8464, 1.0010610702153326)


<B>3</B> Continuation: Suppose now that the system operates under LIFO (Last-In-First-Out of
line): Arriving customers join the front of the queue instead of the end of the queue. We
pointed out in lecure that the stochastic process $\{L(t) : t ≥ 0\}$ is identical to that forFIFO. But delays (in line) of individual customers Dn do change. You are to estimate
E(D2) by using the following algorithm (in words): When customer 2 (C2) arrives at time
t2, we then keep simulating under LIFO until C2 enters service, the time is denoted by
$t^{(s)}_2 \dot T = t^{(s)}_2$
then the (random) time at which the simulation stops. And then output
$D2 = t^{(s)}_2 − t_2$. Repeat 10,000 times so as to use Monte Carlo to estimate E(D2).
Note that D2 > 0 if and only if when C2 arrives at time t2, C1 (who arrived at time t1)
is still in service: under that case, some of the future customers C3, . . . might also arrive
before C2

In [33]:
def lifo_2( lamb,a):
    # customer 1 enters
    entry = arrival(lamb)  
    exit = entry + deperature(a) # when customer 1 will exit
    
    # customer 2 enters
    c = arrival(lamb) 
    entry = entry + arrival(lamb)
    t_2 = entry
    
    # customer 3 enters if before customer 1 leaves
    c = arrival(lamb) 
    entry = entry + c if entry + c < exit else None
    while entry is not None:
        # subsequent customer entries if non customer 2 being served
        exit = entry + deperature(a)
        c = arrival(lamb)
        entry = entry + c if entry + c < exit else None     
    return exit - t_2 if exit > t_2 else 0

print sum([lifo_2(.2,2) for x in range(10000)])/10000

1.12464619632


<B>4.</B> Under the model in (1), suppose we allow customers to become impatient: Upon arrival, ifa customer must wait in the line, then (independent of each other), the customer will leavewithout being served if after an exponential amount of time J that has rate α = 1/4, theyhave not yet entered service. Each arriving customer (who waits in line) gets there own iidsuch exponential. (YOU DO NOT NEED TO DO A SIMULATION/PROGRAMMING
HERE; JUST ANSWER SOME QUESTIONS.)

(a) What are the events if we wish to simulate (as before) ?

* Arrival Time
* Deperature Time (if no one is in the system, also set leaving to 0)
* Leaving Time

(b) Give pseudo-code for the various cases now.
While t < T:

* if arrivial time is the least:
    * add person to system, reset arrival time, generate leave time if n >=1 and add to list
* if deperature time is the least:
    * subtract persom from system
    * reset deperature time(service) if there are people waiting
    * if no one is waiting, set deperature to infinity and pop top of leave times
* if top leave time is the least
    * subtract person from system
    * set next soonest leave time to top if exists
    * else set top to infinity

rest of algo is the same
    